## Problem 4b

Modify the finite system DMRG code provided in the lecture to solve a one-dimensional Hubbard model with $10$ sites.  

$$H=-t\sum_{i=1}^{L-1} \sum_{\sigma} \left(c_{i,\sigma}^{\dagger} c_{i+1,\sigma} + h.c. \right) + \sum_{i=1}^L U n_{i,\uparrow} n_{i,\downarrow} - \mu (n_{i,\uparrow}+n_{i,\downarrow}).$$

You can use the parameters $(t=-1, U=0.5, \mu=0)$. The reference ground state energy is $-10.8564601187$.


Note: 

1) You can validate the correctness of your code by setting $U=0$, and the energy should agree with that obtained from diagonalizing the quadratic part. 

2) Due to the length of the code, you may want to write a separate Julia / python code instead of using the notebook directly.

3) Some useful convention:

- Each site has dimension 4, with the convention following the Kronecker product: spin up $\otimes$ spin down, i.e. the spin down occupies the inner dimension (row-major)

- For creation operators $c_{i,\sigma}^\dagger$, larger $i$ occupies the inner dimension (i.e. row-major, consistent with Kronecker product)

- The Jordan-Wigner transformation is

$$c_{i,\sigma} = \sigma_z\otimes\cdots \otimes \sigma_z \otimes A_{\sigma} \otimes I\otimes \cdots\otimes I,$$

i.e. the pairity accumulates towards the left direction. This means the convention for creating a determinant is e.g. $c_{1,\sigma}^{\dagger}c_{2,\sigma}^{\dagger}c_{3,\sigma}^{\dagger}\vert 0\rangle$.

After the JW transformation, the 1D Hubbard model becomes again a local Hamiltonian

$$H=-\sum_{i=1}^{N-1} \sum_{\sigma} \left[(a_{i,\sigma}^{\dagger} \sigma^{z,\uparrow,\downarrow}_{i}) a_{i+1,\sigma} + (\sigma^{z,\uparrow,\downarrow}_{i} a_{i,\sigma}) a_{i+1,\sigma}^{\dagger}  \right]+  \sum_{i=1}^L U n_{i,\uparrow} n_{i,\downarrow} - \mu (n_{i,\uparrow}+n_{i,\downarrow}) n_{i,\downarrow}.$$

Here 

$$a_{i,\sigma} =  I\otimes \cdots\otimes I \otimes A_{\sigma} \otimes I\otimes \cdots\otimes I$$
$$a^{\dagger}_{i,\sigma} =  I\otimes \cdots\otimes I \otimes A^{\dagger}_{\sigma} \otimes I\otimes \cdots\otimes I$$
$$\sigma^{z,\uparrow,\downarrow}_{i} = I\otimes \cdots\otimes I \otimes \sigma^{z,\uparrow,\downarrow} \otimes I\otimes \cdots\otimes I$$

are the bosonic annihilation and creation operator and commute (hence note the reversed order in the term $(\sigma^{z,\uparrow,\downarrow}_{i} a_{i,\sigma}) a_{i+1,\sigma}^{\dagger}$. This is OK since operators on different sites now commute), while the definition of $n_{i,\sigma}$ are the same.

4) Some references (you do not necessarily need to refer to these)

    Eric Jeckelmann, Density-Matrix Renormalization Group Algorithms, 2008

as well as the implementation from

https://simple-dmrg.readthedocs.io/en/latest/

https://dmrg101-tutorial.readthedocs.io/en/latest/index.html

https://github.com/Andrew-Shen/DMRG/tree/master/DMRG-Hubbard


**The following implementation is provided by Yu Tong**


In [2]:
import numpy as np
import numpy.linalg as la
import MPS
import MPO
import Utils

L = 10
D = 20  # bond dimension
t = -1.0
U = 0.5


#Generate the Hubbard Hamiltonian in Matrix Product Operator representation:
mpo = Utils.generate_hubbard_mpo(t,U,0.0,L)

#random initial guess
mps1 = Utils.generate_random_mps(L,4,D)

#Optimize
gs_energy = mpo.variation_ground_state(mps1,20,1e-4)

('At iteration', 0, 'the energy is', -10.776815905697175)
('At iteration', 1, 'the energy is', -10.852159247612283)
('At iteration', 2, 'the energy is', -10.85425324124937)
('At iteration', 3, 'the energy is', -10.854437364348053)
('At iteration', 4, 'the energy is', -10.854473053676433)


**Code validation for the non-interacting system**

In [5]:
L = 10
D = 20
t = -1.0
U = 0.0
mpo = Utils.generate_hubbard_mpo(t,U,0.0,L)
mps1 = Utils.generate_random_mps(L,4,D)
gs_energy = mpo.variation_ground_state(mps1,20,1e-4)

h1e = -t*( np.diag(np.ones(L-1),1) + np.diag(np.ones(L-1),-1) )
eigvals, eigvecs = la.eigh(h1e)
ref_energy = 0.0
for ev in eigvals:
    if ev < 0.0:
        ref_energy += ev
ref_energy *= 2
print("The exact mean-field ground state energy is",ref_energy)

('At iteration', 0, 'the energy is', -11.975162136975287)
('At iteration', 1, 'the energy is', -12.050585702179156)
('At iteration', 2, 'the energy is', -12.051374396038375)
('At iteration', 3, 'the energy is', -12.051395795824503)
('The exact mean-field ground state energy is', -12.05334836666454)
